In [289]:
# Helper Website: https://towardsdatascience.com/random-forest-in-python-24d0893d51c0

import torch
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import IPython
import os
import zipfile
import os
import cv2
import string
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch import optim as optim, functional as F
from torchvision.transforms import ToTensor
from torch.nn.utils.rnn import pad_sequence
from matplotlib import pyplot as plt, transforms


device = torch.device('cpu')

df = pd.read_csv('/Users/devpatelio/Downloads/Coding/Python/ArcGIS/data.csv')
df.head()

,AcresBurned,Latitude,Longitude,Started,UniqueID,StationDistance,StationID,Total ETo (in),Total Precip (in),Avg Sol Rad (Ly/day),Avg Vap Pres (mBars),Avg Max Air Temp (F),Avg Min Air Temp (F),Avg Air Temp (F),Avg Max Rel Hum (%),Avg Min Rel Hum (%),Avg Rel Hum (%),Avg Dew Point (F),Avg Wind Speed (mph),Avg Soil Temp (F)
0,281893,34.415210,-119.091240,Dec 2017,d28bc34e-73a8-454d-9e55-dea7bdd40bee,6.319871,198,3.52,0.00,266,6.6,76.9,44.2,59.5,71,21,42.0,32.3,4.7,57.9
1,281893,34.415210,-119.091240,Dec 2017,d28bc34e-73a8-454d-9e55-dea7bdd40bee,6.319871,198,3.52,0.00,266,6.6,76.9,44.2,59.5,71,21,42.0,32.3,4.7,57.9
2,132127,36.459940,-121.899380,Jul 2016,977ac7ce-4204-4cd3-9f89-727142165261,5.713349,210,5.00,0.01,581,14.0,65.5,50.6,57.1,100,74,88.0,53.6,3.6,68.3
3,97717,38.805437,-120.629714,Sep 2014,3df6ecf7-f095-483d-b1c4-2dc969d61261,8.025295,13,6.01,1.05,462,9.3,82.1,59.7,70.6,54,28,40.0,41.8,5.4,64.1
4,96949,34.235000,-118.701280,Nov 2018,92d7aeea-7abf-47a4-bf76-a4fe0fac65c6,4.155006,219,3.26,0.87,298,7.4,76.1,51.0,62.4,66,23,42.0,31.4,3.1,65.2


In [290]:
def normalize(vals, iter):
    assert type(vals) is list
    mm = min(iter)
    mx = max(iter)

    return [(i-mm)/(mx-mm) for i in vals]

bias_vals = df['AcresBurned'].to_list()
avg = normalize([sum(bias_vals)/len(bias_vals)], bias_vals)
bias_vals = normalize(bias_vals, bias_vals)

print(bias_vals)
print(avg)

df['RiskLevel'] = bias_vals

[1.0, 1.0, 0.4686944583390981, 0.34662253488149336, 0.3438980002341397, 0.3438980002341397, 0.2006009585537262, 0.2006009585537262, 0.17351170521102727, 0.17351170521102727, 0.17351170521102727, 0.11121280815089948, 0.0820411305399758, 0.06534626068262364, 0.05537403816477049, 0.04437302001184889, 0.03665705274883551, 0.035436688271375, 0.03540121255982092, 0.031179602884884864, 0.028735326358808443, 0.025485751180454302, 0.022981165944735937, 0.021423782207511628, 0.0203204875781796, 0.020295654580091742, 0.017078007542136276, 0.01615563904173008, 0.0160385691936016, 0.015006225987377742, 0.015006225987377742, 0.013714910086809065, 0.011955314793726475, 0.011196134566469068, 0.011054231720252728, 0.010408573769968392, 0.008641883334574984, 0.008013963240067687, 0.007634373126438983, 0.007307996580141406, 0.007258330583965688, 0.007002905460776279, 0.006889383183803209, 0.006797146333762589, 0.00670490948372197, 0.0063714377951135754, 0.006296938800849998, 0.006194059237343153, 0.00585

In [291]:
high_risk = [i for i in bias_vals if i > avg[0]]
low_risk = [i for i in bias_vals if i < avg[0]]

print(len(high_risk), len(low_risk))
print(low_risk)

low_avg = sum(low_risk)/len(low_risk)

print(low_risk)
print(low_avg)

medium_risk = [i for i in low_risk if i >= low_avg]
low_risk = [i for i in low_risk if i < low_avg]

31 349
[0.013714910086809065, 0.011955314793726475, 0.011196134566469068, 0.011054231720252728, 0.010408573769968392, 0.008641883334574984, 0.008013963240067687, 0.007634373126438983, 0.007307996580141406, 0.007258330583965688, 0.007002905460776279, 0.006889383183803209, 0.006797146333762589, 0.00670490948372197, 0.0063714377951135754, 0.006296938800849998, 0.006194059237343153, 0.0058534924064239414, 0.005456164437018195, 0.005296523735024815, 0.004753745348247323, 0.004753745348247323, 0.004438011515415971, 0.003898780699793886, 0.003668188574692337, 0.0030402684801850415, 0.002880627778191661, 0.002660678366556337, 0.0026464880819347036, 0.0026074647992252106, 0.0024903949510967317, 0.002341396962569577, 0.0022065892586640556, 0.0021108048374680276, 0.002093066981690985, 0.0019156884239205627, 0.0018092612892583093, 0.0018021661469474924, 0.0017170244392176897, 0.0017099292969068728, 0.001688643869974422, 0.0015964070199338024, 0.0015609313083797178, 0.0015538361660689009, 0.0015396

In [292]:
scaled_df = df.copy()
scaled_df['AcresBurned'] = bias_vals
scaled_df.head()

for column in scaled_df:
    try:
        scaled_df[column] = normalize(scaled_df[column].to_list(), scaled_df[column].to_list())
    except:
        pass

In [293]:
scaled_df.head()

,AcresBurned,Latitude,Longitude,Started,UniqueID,StationDistance,StationID,Total ETo (in),Total Precip (in),Avg Sol Rad (Ly/day),...,Avg Max Air Temp (F),Avg Min Air Temp (F),Avg Air Temp (F),Avg Max Rel Hum (%),Avg Min Rel Hum (%),Avg Rel Hum (%),Avg Dew Point (F),Avg Wind Speed (mph),Avg Soil Temp (F),RiskLevel
0,1.000000,0.204727,0.475963,Dec 2017,d28bc34e-73a8-454d-9e55-dea7bdd40bee,0.606650,0.756098,0.180169,0.000000,0.187943,...,0.364017,0.395470,0.356877,0.560606,0.147059,0.287671,0.373585,0.493333,0.316832,1.000000
1,1.000000,0.204727,0.475963,Dec 2017,d28bc34e-73a8-454d-9e55-dea7bdd40bee,0.606650,0.756098,0.180169,0.000000,0.187943,...,0.364017,0.395470,0.356877,0.560606,0.147059,0.287671,0.373585,0.493333,0.316832,1.000000
2,0.468694,0.430070,0.150293,Jul 2016,977ac7ce-4204-4cd3-9f89-727142165261,0.541699,0.804878,0.359129,0.000730,0.411348,...,0.125523,0.506969,0.312268,1.000000,0.926471,0.917808,0.775472,0.346667,0.574257,0.468694
3,0.346623,0.688560,0.297541,Sep 2014,3df6ecf7-f095-483d-b1c4-2dc969d61261,0.789279,0.004065,0.481258,0.076642,0.326950,...,0.472803,0.665505,0.563197,0.303030,0.250000,0.260274,0.552830,0.586667,0.470297,0.346623
4,0.343898,0.184867,0.521188,Nov 2018,92d7aeea-7abf-47a4-bf76-a4fe0fac65c6,0.374821,0.841463,0.148730,0.063504,0.210638,...,0.347280,0.513937,0.410781,0.484848,0.176471,0.287671,0.356604,0.280000,0.497525,0.343898


In [294]:
data_df = scaled_df.copy()
data_df = [data_df[label] for label in data_df if label not in ['Started', 'UniqueID']]

data_df = pd.DataFrame(data_df).transpose()
data_df.head()

,AcresBurned,Latitude,Longitude,StationDistance,StationID,Total ETo (in),Total Precip (in),Avg Sol Rad (Ly/day),Avg Vap Pres (mBars),Avg Max Air Temp (F),Avg Min Air Temp (F),Avg Air Temp (F),Avg Max Rel Hum (%),Avg Min Rel Hum (%),Avg Rel Hum (%),Avg Dew Point (F),Avg Wind Speed (mph),Avg Soil Temp (F),RiskLevel
0,1.000000,0.204727,0.475963,0.606650,0.756098,0.180169,0.000000,0.187943,0.204301,0.364017,0.395470,0.356877,0.560606,0.147059,0.287671,0.373585,0.493333,0.316832,1.000000
1,1.000000,0.204727,0.475963,0.606650,0.756098,0.180169,0.000000,0.187943,0.204301,0.364017,0.395470,0.356877,0.560606,0.147059,0.287671,0.373585,0.493333,0.316832,1.000000
2,0.468694,0.430070,0.150293,0.541699,0.804878,0.359129,0.000730,0.411348,0.602151,0.125523,0.506969,0.312268,1.000000,0.926471,0.917808,0.775472,0.346667,0.574257,0.468694
3,0.346623,0.688560,0.297541,0.789279,0.004065,0.481258,0.076642,0.326950,0.349462,0.472803,0.665505,0.563197,0.303030,0.250000,0.260274,0.552830,0.586667,0.470297,0.346623
4,0.343898,0.184867,0.521188,0.374821,0.841463,0.148730,0.063504,0.210638,0.247312,0.347280,0.513937,0.410781,0.484848,0.176471,0.287671,0.356604,0.280000,0.497525,0.343898


In [295]:
data_df.isnull().values.any()
data_df.isnull().sum().sum()

print(data_df.to_string())


     AcresBurned  Latitude  Longitude  StationDistance  StationID  Total ETo (in)  Total Precip (in)  Avg Sol Rad (Ly/day)  Avg Vap Pres (mBars)  Avg Max Air Temp (F)  Avg Min Air Temp (F)  Avg Air Temp (F)  Avg Max Rel Hum (%)  Avg Min Rel Hum (%)  Avg Rel Hum (%)  Avg Dew Point (F)  Avg Wind Speed (mph)  Avg Soil Temp (F)  RiskLevel
0       1.000000  0.204727   0.475963         0.606650   0.756098        0.180169           0.000000              0.187943              0.204301              0.364017              0.395470          0.356877             0.560606             0.147059         0.287671           0.373585              0.493333           0.316832   1.000000
1       1.000000  0.204727   0.475963         0.606650   0.756098        0.180169           0.000000              0.187943              0.204301              0.364017              0.395470          0.356877             0.560606             0.147059         0.287671           0.373585              0.493333           0.316832 

In [296]:
data_df.replace(np.nan, 0)
data_df.fillna(0)
data_df.isnull().sum().sum()
data_df.round(6)


,AcresBurned,Latitude,Longitude,StationDistance,StationID,Total ETo (in),Total Precip (in),Avg Sol Rad (Ly/day),Avg Vap Pres (mBars),Avg Max Air Temp (F),Avg Min Air Temp (F),Avg Air Temp (F),Avg Max Rel Hum (%),Avg Min Rel Hum (%),Avg Rel Hum (%),Avg Dew Point (F),Avg Wind Speed (mph),Avg Soil Temp (F),RiskLevel
0,1.000000,0.204727,0.475963,0.606650,0.756098,0.180169,0.000000,0.187943,0.204301,0.364017,0.395470,0.356877,0.560606,0.147059,0.287671,0.373585,0.493333,0.316832,1.000000
1,1.000000,0.204727,0.475963,0.606650,0.756098,0.180169,0.000000,0.187943,0.204301,0.364017,0.395470,0.356877,0.560606,0.147059,0.287671,0.373585,0.493333,0.316832,1.000000
2,0.468694,0.430070,0.150293,0.541699,0.804878,0.359129,0.000730,0.411348,0.602151,0.125523,0.506969,0.312268,1.000000,0.926471,0.917808,0.775472,0.346667,0.574257,0.468694
3,0.346623,0.688560,0.297541,0.789279,0.004065,0.481258,0.076642,0.326950,0.349462,0.472803,0.665505,0.563197,0.303030,0.250000,0.260274,0.552830,0.586667,0.470297,0.346623
4,0.343898,0.184867,0.521188,0.374821,0.841463,0.148730,0.063504,0.210638,0.247312,0.347280,0.513937,0.410781,0.484848,0.176471,0.287671,0.356604,0.280000,0.497525,0.343898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,0.000000,0.626812,0.087122,0.615110,0.394309,0.457074,0.000000,0.416312,0.623656,0.403766,0.426829,0.405204,0.984848,0.588235,0.753425,0.786792,0.293333,0.522277,0.000000
376,0.000000,0.098370,0.692728,0.203410,0.203252,0.544135,0.001460,0.378014,0.650538,0.447699,0.594077,0.470260,0.969697,0.397059,0.671233,0.805660,0.240000,0.596535,0.000000
377,0.000000,0.735575,0.210991,0.400711,0.292683,0.581620,0.018248,0.438298,0.575269,0.439331,0.571429,0.513011,0.696970,0.411765,0.506849,0.745283,0.466667,0.603960,0.000000
378,0.000000,0.108189,0.822600,0.341111,0.504065,0.764208,0.048905,0.475887,0.456989,0.665272,0.681185,0.667286,0.454545,0.132353,0.232877,0.660377,0.466667,0.759901,0.000000


In [297]:
print(data_df.to_string())

     AcresBurned  Latitude  Longitude  StationDistance  StationID  Total ETo (in)  Total Precip (in)  Avg Sol Rad (Ly/day)  Avg Vap Pres (mBars)  Avg Max Air Temp (F)  Avg Min Air Temp (F)  Avg Air Temp (F)  Avg Max Rel Hum (%)  Avg Min Rel Hum (%)  Avg Rel Hum (%)  Avg Dew Point (F)  Avg Wind Speed (mph)  Avg Soil Temp (F)  RiskLevel
0       1.000000  0.204727   0.475963         0.606650   0.756098        0.180169           0.000000              0.187943              0.204301              0.364017              0.395470          0.356877             0.560606             0.147059         0.287671           0.373585              0.493333           0.316832   1.000000
1       1.000000  0.204727   0.475963         0.606650   0.756098        0.180169           0.000000              0.187943              0.204301              0.364017              0.395470          0.356877             0.560606             0.147059         0.287671           0.373585              0.493333           0.316832 

In [298]:
import numpy as np

risks = np.array(data_df['RiskLevel'])
data_df = data_df.drop('RiskLevel', axis=1)
data_df = data_df.drop('AcresBurned', axis=1)
data_df = data_df.drop('StationID', axis=1)

feat_names = list(data_df.columns)
data_df = np.array(data_df)

In [299]:
from sklearn.model_selection import train_test_split
train_x, val_x, train_y, val_y = train_test_split(data_df, risks, test_size=0.2)

In [300]:
def check_nan(dataset):
    a = np.isnan(dataset)
    w = []
    for idx, i in enumerate(a):
        for x, g in enumerate(i):
            if g == True:
#                 print(dataset[idx][x])
                dataset[idx][x] = 0
    if True not in np.isnan(dataset):
        print('Complete')
    else:
        print('Incomplete')

check_nan(train_x)
check_nan(val_x)

Complete
Complete


In [301]:
from sklearn.ensemble import RandomForestRegressor
rfregression = RandomForestRegressor(n_estimators = 1000)

rfregression.fit(train_x, train_y)

RandomForestRegressor(n_estimators=1000)

In [302]:
preds = rfregression.predict(val_x)
errors = abs(preds-val_y)
print(errors)
print(np.mean(errors))

for idx, i in enumerate(val_y):
    if i == float(0):
        val_y[idx] += 0.000000001

print(val_y)

print(np.mean(val_y))

acc = 100 - np.mean(errors)/np.mean(val_y)
print(f"Accuracy: {acc}%")

[2.01768766e-03 2.95561839e-01 7.67529968e-03 9.34660834e-04
 5.92883395e-05 4.36002644e-04 1.24454295e-03 1.97401928e-01
 1.15977232e-02 5.63653301e-04 3.17701138e-03 1.04940880e-03
 2.76213890e-03 4.38029904e-04 1.25554592e-03 2.98494743e-01
 3.86496800e-03 3.22740286e-05 7.50283274e-02 1.02027082e-03
 3.59355477e-04 5.67663179e-03 1.78064658e-03 5.22802014e-03
 9.63209026e-04 2.57574951e-04 6.50034707e-04 5.06364875e-03
 2.63540902e-03 4.44060124e-04 4.80015467e-03 6.75968398e-04
 4.79757911e-03 1.28620740e-04 1.71192835e-03 3.17042887e-04
 1.73566089e-02 5.02477978e-05 2.14529610e-04 8.30273553e-05
 5.09545450e-02 2.20271354e-04 4.30383084e-02 6.92559916e-04
 7.25124431e-04 2.88594913e-04 1.03074562e-03 2.57574951e-04
 2.06388466e-03 7.74326937e-04 8.04758169e-02 6.99199668e-03
 8.49941678e-02 2.20875328e-04 1.24750499e-03 9.11268742e-03
 9.29781505e-03 3.45638559e-01 1.46659075e-03 1.01947297e-02
 1.07347375e-02 8.39233307e-03 8.98597503e-02 1.01621737e-02
 1.61274495e-02 6.069214

In [303]:
from sklearn.tree import export_graphviz
import pydot
tree = rfregression.estimators_[5]
export_graphviz(tree, out_file = 'tree.dot', feature_names=feat_names, rounded=True, precision=1)
(graph, ) = pydot.graph_from_dot_file('/Users/devpatelio/Downloads/Coding/Python/ArcGIS/tree.dot')
# refer to tree.png file for visual rep of dot file

importances = list(rfregression.feature_importances_)
feat_importance = [(data_df, round(importance, 2)) for feature, importance in zip(feat_names, importances)]
feat_importance = sorted(feat_importance, key=lambda x: x[1], reverse=True)

printed = [f"Name: {feat_names[idx]}, Importance: {feat_importance[idx][-1]}" for idx in range(len(feat_names))]
print(printed)


#After observing that our model can effectively map this, we need to now remove the acresburned column and train the model accordingly

['Name: Latitude, Importance: 0.15', 'Name: Longitude, Importance: 0.12', 'Name: StationDistance, Importance: 0.1', 'Name: Total ETo (in), Importance: 0.09', 'Name: Total Precip (in), Importance: 0.09', 'Name: Avg Sol Rad (Ly/day), Importance: 0.08', 'Name: Avg Vap Pres (mBars), Importance: 0.06', 'Name: Avg Max Air Temp (F), Importance: 0.06', 'Name: Avg Min Air Temp (F), Importance: 0.04', 'Name: Avg Air Temp (F), Importance: 0.04', 'Name: Avg Max Rel Hum (%), Importance: 0.04', 'Name: Avg Min Rel Hum (%), Importance: 0.04', 'Name: Avg Rel Hum (%), Importance: 0.03', 'Name: Avg Dew Point (F), Importance: 0.03', 'Name: Avg Wind Speed (mph), Importance: 0.02', 'Name: Avg Soil Temp (F), Importance: 0.01']
